In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from pandas.io.json import json_normalize
import numpy as np
from geopy.distance import geodesic

``Importo collections sobre los requisitos en San Francisco``

In [2]:
dbName = "Crunchbase_Dataset"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL)
db = client.get_database()

In [3]:
curse = db.gamers_companies.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
gamer_companies = list(curse)
df= pd.DataFrame(gamer_companies)
df.head()

,_id,name,latitude,longitude,location
0,5ef9c53e45de338078b9ab82,Ustream,37.392936,-122.079480,"{'type': 'Point', 'coordinates': [-122.07948, ..."
1,5ef9c53e45de338078b9ab83,Revision3,37.757758,-122.388243,"{'type': 'Point', 'coordinates': [-122.388243,..."
2,5ef9c53e45de338078b9ab84,CastTV,37.780716,-122.393913,"{'type': 'Point', 'coordinates': [-122.393913,..."
3,5ef9c53e45de338078b9ab85,Metacafe,37.437328,-122.159928,"{'type': 'Point', 'coordinates': [-122.159928,..."
4,5ef9c53e45de338078b9ab86,Mevio,37.782263,-122.392142,"{'type': 'Point', 'coordinates': [-122.392142,..."


In [4]:
curse = db.vegan.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
vegan = list(curse)
df_vegan= pd.DataFrame(vegan)
df_vegan.head()

,_id,name,latitude,longitude,location
0,5ef9c61445de338078b9abae,Ananda Fuara,37.777693,-122.416353,"{'type': 'Point', 'coordinates': [-122.4163533..."
1,5ef9c61445de338078b9abaf,Golden Era Vegan,37.781495,-122.416822,"{'type': 'Point', 'coordinates': [-122.4168223..."
2,5ef9c61445de338078b9abb0,The Flying Falafel,37.781323,-122.411138,"{'type': 'Point', 'coordinates': [-122.411138,..."
3,5ef9c61445de338078b9abb1,Ike's Sandwiches,37.785146,-122.419629,"{'type': 'Point', 'coordinates': [-122.4196290..."
4,5ef9c61445de338078b9abb2,Shizen,37.768190,-122.421627,"{'type': 'Point', 'coordinates': [-122.4216272..."


In [5]:
curse = db.airports.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
airports = list(curse)
df_airports= pd.DataFrame(airports)
df_airports.head()

,_id,name,latitude,longitude,location
0,5ef9c51645de338078b9ab76,Design To Win Foundation,37.790943,-122.408499,"{'type': 'Point', 'coordinates': [-122.4084994..."
1,5ef9c51645de338078b9ab77,West Sunset RC Flying Field,37.748486,-122.497641,"{'type': 'Point', 'coordinates': [-122.4976407..."
2,5ef9c51645de338078b9ab78,SFO,37.785729,-122.410446,"{'type': 'Point', 'coordinates': [-122.4104463..."
3,5ef9c51645de338078b9ab79,MUrgency Inc. (Airport Assistance; Tech Innova...,37.766902,-122.419558,"{'type': 'Point', 'coordinates': [-122.419558,..."
4,5ef9c51645de338078b9ab7a,UCSF Helipad,37.766611,-122.390381,"{'type': 'Point', 'coordinates': [-122.3903809..."


In [6]:
curse = db.clubs.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
clubs = list(curse)
df_clubs= pd.DataFrame(clubs)
df_clubs.head()

,_id,name,latitude,longitude,location
0,5ef9c4e645de338078b9ab62,Zero Zero,37.781636,-122.402083,"{'type': 'Point', 'coordinates': [-122.4070662..."
1,5ef9c4e645de338078b9ab63,Smuggler's Cove,37.779403,-122.423372,"{'type': 'Point', 'coordinates': [-122.4070662..."
2,5ef9c4e645de338078b9ab64,The Valencia Room,37.762506,-122.421412,"{'type': 'Point', 'coordinates': [-122.4070662..."
3,5ef9c4e645de338078b9ab65,The Grand,37.779100,-122.397900,"{'type': 'Point', 'coordinates': [-122.4070662..."
4,5ef9c4e645de338078b9ab66,Edinburgh Castle Pub,37.786127,-122.418957,"{'type': 'Point', 'coordinates': [-122.4070662..."


In [7]:
curse = db.schools.find({},{"name":1,"location":1,"latitude":1,"longitude":1})
schools = list(curse)
df_schools= pd.DataFrame(schools)
df_schools.head()

,_id,name,latitude,longitude,location
0,5ef9c5db45de338078b9ab90,John Swett Alternative Elementary School,37.780412,-122.422916,"{'type': 'Point', 'coordinates': [-122.422916,..."
1,5ef9c5db45de338078b9ab91,Chinese American International School,37.775275,-122.422111,"{'type': 'Point', 'coordinates': [-122.4221108..."
2,5ef9c5db45de338078b9ab92,Mission Dolores Park,37.759990,-122.427118,"{'type': 'Point', 'coordinates': [-122.4271184..."
3,5ef9c5db45de338078b9ab93,C5 Children's School,37.778864,-122.414584,"{'type': 'Point', 'coordinates': [-122.4145837..."
4,5ef9c5db45de338078b9ab94,John Muir Elementary School,37.773624,-122.428363,"{'type': 'Point', 'coordinates': [-122.4283626..."


Buscaremos el punto exacto desde este punto (37.78131,-122.390025) ya que en el mapa hemos visto que es donde podemos encontrar mayor numero de empresas de gaming, más restaurantes veganos, clubs y colegios. 
 


In [8]:
def geocode(address):
    res = requests.get(f"https://geocode.xyz/{address}", params={"json":1})
    data = res.json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

In [9]:
def getlocationeNear(address, maxDist=600):
    point = geocode(address)
    return {
       "location": {
         "$near": {
           "$geometry": point,
           "$maxDistance": maxDist,
         }
       }
    }

In [10]:
query=getlocationeNear("Colin P Kelly Jr St, San Francisco, CA, United States of America") 
query

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-122.39131, 37.78245]},
   '$maxDistance': 600}}}

In [11]:
cur = db.gamers_companies.find(query, {"_id":0})
print(cur.count())
result = list(cur)
result

5


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


[{'name': 'Mevio',
  'latitude': 37.782263,
  'longitude': -122.392142,
  'Requirement': 'gamer_companies',
  'location': {'type': 'Point', 'coordinates': [-122.392142, 37.782263]}},
 {'name': 'Shattered Reality Interactive',
  'latitude': 37.783046,
  'longitude': -122.394067,
  'Requirement': 'gamer_companies',
  'location': {'type': 'Point', 'coordinates': [-122.394067, 37.783046]}},
 {'name': 'Shattered Reality Interactive',
  'latitude': 37.783046,
  'longitude': -122.394067,
  'Requirement': 'gamer_companies',
  'location': {'type': 'Point', 'coordinates': [-122.394067, 37.783046]}},
 {'name': 'CastTV',
  'latitude': 37.780716,
  'longitude': -122.393913,
  'Requirement': 'gamer_companies',
  'location': {'type': 'Point', 'coordinates': [-122.393913, 37.780716]}},
 {'name': 'Rupture',
  'latitude': 37.783898,
  'longitude': -122.395234,
  'Requirement': 'gamer_companies',
  'location': {'type': 'Point', 'coordinates': [-122.395234, 37.783898]}}]

In [14]:
cur = db.schools.find(query, {"_id":0})
print(cur.count())
result = list(cur)
result

0


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


[]